In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import nltk
from nltk.corpus import stopwords
from openai import OpenAI
from dotenv import load_dotenv
import re

load_dotenv()
client = OpenAI(api_key=os.environ["OPENROUTER_API_KEY"], base_url="https://openrouter.ai/api/v1")

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
website_df = pd.read_csv("website_data.csv")

website_df.head()

[nltk_data] Downloading package stopwords to /Users/vijay/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/rw/s7c9z30x7fj_2gcngv0yhv6w0000gn/T/ipykernel_84803/1791604069.py:18: DtypeWarning: Columns (300,304,325,328) have mixed types. Specify dtype option on import or set low_memory=False.
  website_df = pd.read_csv("website_data.csv")


,Unnamed: 0,std_name,company_name,company_website_link,Made_in_USA_related_info_link,notes,list_order,list_title,list_link,Title_List_Name,...,1997-07,1997-06,1997-05,1997-04,1997-03,1997-02,1997-01,1996-12,1996-11,1996-10
0,0,11industries,11 Industries,https://11industries.com/,NaN,"Wallets, Dopp Kits, Bags | ""All our products ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,11wellsspirits,11 Wells Spirits,https://www.11wells.com,NaN,NaN,194.0,American Rum Producers,https://www.robsrum.com/americanrum/,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,14throse,14th Rose,https://14throse.com/,NaN,"Women?s Purses, Totes | ""14th Rose Purses are ...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1919cookware,1919 Cookware,https://www.regalware.com,NaN,NaN,373.0,NaN,https://usamadeproducts.biz/appliances-kitchen...,USA Made Cookware List | 29 Brands & Manufactu...,...,NaN,NaN,NaN,NaN,NaN,NaN,"\nWelcome to Regal Ware, Inc.\nA global leader...","\nWelcome to Regal Ware, Inc.\nA global leader...",NaN,NaN
4,4,20degeesbelow,20 Degees below,https://20below.net,NaN,NaN,352.0,NaN,https://americansworking.com/clothing.html,Clothing - americans-working,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

def propagate_str_value_forward(df, min_length=1):
    df = df.copy()
    # Replace empty strings with NaN
    df.replace('', np.nan, inplace=True)  
    # Replace strings with length less than min_length with NaN
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) and len(x) < min_length else x)
    # Forward fill NaN values
    df = df.T.ffill(axis=0).T  
    # Fill remaining NaN with empty strings
    df.fillna('', inplace=True)  
    return df

def remove_stopwords(text):
    if isinstance(text, str):
        return ' '.join([word for word in text.split() if word.lower() not in stop_words])
    return text

selected_cols = reversed([col for col in website_df.columns if col.startswith("1") or col.startswith("2")])
website_df = website_df[selected_cols]
website_df = propagate_str_value_forward(website_df)

for col in tqdm(website_df.select_dtypes(include=['object']).columns):
    website_df[col] = website_df[col].apply(remove_stopwords)

In [4]:
def classify_text_with_openai(text):
    prompt = f"Classify the text into the likelist SIC code for the business. If you're unsure, match it to the closest SIC code from the segment of text you see. \n\nText: {text[:1000]}\n\n SIC Code:"
    
    try:
        response = client.chat.completions.create(
            model="google/gemma-2-9b-it",
            messages=[
                {"role": "system", "content": "You are a SIC classification assistant."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=30,
            temperature=0.3
        )
        
        classification = response.choices[0].message.content.strip()

        sic_code_match = re.search(r'\b\d{4}\b', classification)
        if sic_code_match:
            classification = sic_code_match.group(0)
        else:
            # print("Couldn't match SIC code in: ", classification, text[:1000], len(text))
            digits = ''.join(re.findall(r'\d+', classification))
            return digits
            # if len(digits) >= 4:
            #     classification = digits[:4]
            # else:
            #     classification = "0000"

        return classification
    except Exception as e:
        print(f"Error processing text: {e}")
        return "0000"  # Default for errors

In [5]:
website_df.iloc[:, -1]

0       Gifts Men | 11 Industries | Gentleman's Genera...
1       Distillery | 11 Wells Distillery | Saint Paul ...
2       14th Rose - Handbags, Purses, Woman's Bags Fre...
3       Home - Regal Ware Work Core Values Story Leade...
4       Custom Swim Parkas Wholesale Fleece Jackets | ...
                              ...                        
5184                                                   ��
5185    Home - Xuron Corporation, original inventor Mi...
5186    Yellow 108 – Yellow 108 | Sustainable Headwear...
5187    Cast Touring - Standard Alpine Touring Freerid...
5188    handmade ultra light backpacking gear – YAR.ge...
Name: 2023-10, Length: 5189, dtype: object

In [6]:
import concurrent.futures
import os
import json
from tqdm import tqdm
import threading
import time

# Function to process a single row
def process_row(row_data):
    i, row = row_data
    text = row[-1]
    classification = classify_text_with_openai(text)
    return i, classification

# Load existing classifications
website_classifications = []
if os.path.exists("website_classifications.json"):
    with open("website_classifications.json", "r") as f:
        website_classifications = json.load(f)

# Initialize or prepare for processing
if not website_classifications or len(website_classifications) != len(website_df):
    # If we need to start fresh or sizes don't match
    if len(website_classifications) != len(website_df):
        website_classifications = [None] * len(website_df)
    
    # Determine which rows need processing
    tasks = []
    for i, row in enumerate(website_df.itertuples(index=False)):
        if i >= len(website_classifications) or website_classifications[i] is None:
            tasks.append((i, row))
    
    # Set up saving mechanism
    save_lock = threading.Lock()
    last_save_time = time.time()
    
    # Number of concurrent workers - adjust based on API rate limits
    max_workers = 5
    
    # Process in parallel
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit all tasks
        futures = {executor.submit(process_row, task): task[0] for task in tasks}
        
        # Process results as they complete
        for future in tqdm(concurrent.futures.as_completed(futures), total=len(tasks)):
            idx = futures[future]
            try:
                i, classification = future.result()
                website_classifications[i] = classification
                
                # Save periodically (every 20 seconds)
                current_time = time.time()
                if current_time - last_save_time > 20:
                    with save_lock:
                        with open("website_classifications.json", "w") as f:
                            json.dump(website_classifications, f)
                    last_save_time = current_time
                    
            except Exception as exc:
                print(f'Error processing item {idx}: {exc}')
    
    # Final save
    with open("website_classifications.json", "w") as f:
        json.dump(website_classifications, f)

website_df["category"] = website_classifications
website_df.to_csv("website_data_with_classifications.csv", index=False)

 12%|█▏        | 633/5189 [02:35<23:19,  3.25it/s]

Error processing text: list index out of range


 17%|█▋        | 896/5189 [03:42<23:16,  3.07it/s]

Error processing text: list index out of range


 17%|█▋        | 906/5189 [03:46<32:34,  2.19it/s]

...

 98%|█████████▊| 5097/5189 [21:15<00:19,  4.68it/s]

Error processing text: list index out of range


100%|█████████▉| 5164/5189 [21:30<00:04,  5.89it/s]

Error processing text: list index out of range


100%|██████████| 5189/5189 [21:36<00:00,  4.00it/s]
/var/folders/rw/s7c9z30x7fj_2gcngv0yhv6w0000gn/T/ipykernel_11529/2770931402.py:67: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  website_df["category"] = website_classifications
